In [1]:
from statistics import mode
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, State
import numpy as np
import pandas as pd
import plotly.graph_objs as go

In [ ]:
path = 'https://raw.githubusercontent.com/foazul/Data-visualization-Project_1/main/'

df = pd.read_csv(path + 'CO2_Emission_Continent_2.csv')

df_pivot = pd.read_csv('CO2_Emission_Continent_2.csv') #this dataframe was created based on the original one in order to calculate variations and means

def variation(number1, number2):
    result = []
    year1 = 2000 + number1
    year2 = 2000 + number2
    for i in range(len(df_pivot['continent'])):
        result.append(((df_pivot[year2][i] - df_pivot[year1][i])/df_pivot[year2][i])*100)
    return result



######################################################Interactive Components############################################

continent_options = [dict(label=continent, value=continent) for continent in df['continent'].unique()]


dropdown_country = dcc.Dropdown(
        id='continent_drop',
        options=continent_options,
        value=['Europe', 'North America'],
        multi=True
    )


slider_year = dcc.RangeSlider(
        id='year_slider',
        min=df['year'].min(),
        max=df['year'].max(),
        value=[df['year'].min(), df['year'].max()],
        marks={str(i): '{}'.format(str(i)) for i in
               [2011, 2014, 2017, 2020]},
        step=1,
        pushable=1,
        allowCross=False
    )

radio_var_mean = dcc.RadioItems(
        id='var_mean',
        options=[dict(label='Variation', value=0), dict(label='Mean', value=1)],
        value=0
    )




##################################################APP###################################################################

app = dash.Dash(__name__)

server = app.server

app.layout = html.Div ( [

    html.Div([
        html.H1('Emissions Dashboard'),
    ], id='1st row', className='pretty_box'),
    html.Div([
        html.Div([
            html.Br(),
            html.Label('Continent Choice'),
            dropdown_country,
            html.Br(),
            html.Br(),
            html.Label('Year Slider'),
            slider_year,
            html.Br(),
            html.Br(),
            html.Label('Variation or Mean'),
            radio_var_mean,
            html.Br(),
            html.Br(),
            html.Button('Submit', id='button')
        ], id='Iteraction', style={'width': '40%'}, className='pretty_box'),
        html.Div([
            dcc.Graph(id='bar_graph2')
        ]
        , id='Graph3', style={'width': '50%'}, className='pretty_box')
    ], id='2nd row', style={'display': 'flex'}),
    html.Div([
        html.Div([
            dcc.Graph(id='scatter_graph'),
        ], id='Graph1', style={'width': '50%'}, className='pretty_box'),
        html.Div([
            dcc.Graph(id='bar_graph')
        ], id='Graph2', style={'width': '50%'}, className='pretty_box'),
    ], id='3th row', style={'display': 'flex'})
])

######################################################Callbacks#########################################################


@app.callback(
    [
        Output("scatter_graph", "figure"),
        Output("bar_graph", "figure"),
        Output("bar_graph2", "figure"),
    ],
    [
        Input("button", "n_clicks")
    ],
    [
        State("year_slider", "value"),
        State("continent_drop", "value"),
        State("var_mean", "value"),
    ]
)
def plots(n_clicks, years, continents, calc):
    ############################################Scatter plot##########################################################
    
    data_scatter = []
    for continent in continents:
        df_loc = df.loc[(df['continent'] == continent)]
        data_scatter.append(dict(type='scatter',
                             x=df_loc['year'].unique(), 
                             y=df_loc['CO2_emission'], 
                             name=continent,
                             mode='markers'))

   
    layout_scatter = dict(title=dict(text='CO2 Emissions from 2011 until 2020'),
                      yaxis=dict(title='Emissions'),
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)'
                      )

    
    ############################################Bar Plot######################################################

    #df_loc = df.loc[df['continent'].isin(continents)].groupby('continent').mean().reset_index()
    data_bar = []

    for continent in continents:
        df_bar = df.loc[(df['continent'] == continent)]

        x_bar = df_bar['continent']
        y_bar = df_bar['CO2_emission']

        data_bar.append(dict(type='bar', x=x_bar, y=y_bar, name=continent))


    layout_bar = dict(title=dict(text='Cumulative Emissions in selected years'),
                      yaxis=dict(title='CO2 Emissions'),
                      xaxis=dict(title='Continent'),
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)'
                      )

    data_bar2 = []

    for continent in continents:
        df_bar = df.loc[(df['continent'] == continent)]

        x_bar = df_bar['continent']
        y_bar = df_bar['CO2_emission']

        data_bar2.append(dict(type='bar', x=x_bar, y=y_bar, name=continent))


    layout_bar2 = dict(title=dict(text=['Percentual Variation along the years', 'Mean CO2 Emissions by continent'][calc]),
                      yaxis=dict(title=['Percentual Variation', 'Mean CO2 Emissions'][calc]),
                      xaxis=dict(title='Continent'),
                      paper_bgcolor='rgba(0,0,0,0)',
                      plot_bgcolor='rgba(0,0,0,0)'
                      )                  

                      

    return go.Figure(data=data_scatter, layout=layout_scatter), \
           go.Figure(data=data_bar, layout=layout_bar), \
           go.Figure(data=data_bar2, layout=layout_bar2)



if __name__ == '__main__':
    app.run_server(debug=False, port =8099)

Dash is running on http://127.0.0.1:8099/

Dash is running on http://127.0.0.1:8099/

Dash is running on http://127.0.0.1:8099/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
